# 종목별 네이버 기사 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import math
import re
import pandas as pd
import os
import time
from tqdm import tqdm
from tqdm import trange
from sys import stdout

def crawling_article(key_word):
    Url = 'https://search.naver.com/search.naver?'
    ds = ['2020.01.01', '2021.01.01']
    #ds = ['2020.12.20', '2021.11.15']
    de = ['2020.12.31', '2021.12.06']  # 두번째 날은 어제로 설정

    params = {
        "where" : 'news',
        "sm": 'tab_pge',
        "query": key_word,  # ★인코딩하지말고 넣어라★
        "sort": '0',
        "photo": '0',
        "field": '0',
        "pd": '3',
        "ds": '2020.01.01',
        "de" : '2020.12.31',
        "cluster_rank": '10',
        "mynews" : '1',
        "office_type": '0',
        "office_section_code": '0',
        "news_office_checked" : '1032',
        "nso" : r'so:r,p:from20210101to20211130,a:all',
        "start" : '5000'
    }

    # 경향신문, 국민일보, 동아일보, 한겨례, 매일일보, 조선일보
    news_code = ['1032', '1005', '1020', '1028', '2385', '1023']

    # 크롤링 할 url 저장
    urls = []
    
    page = 1
    
    for i in range(2):
        # 날짜 설정
        params['ds'] = ds[i]
        params['de'] = de[i]
        
        for code in news_code:
            params['news_office_checked'] = code
            raw = requests.get(Url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)

            dom = BeautifulSoup(raw.text, "html.parser")

            try:
                a_tags = dom.select('#main_pack > div.api_sc_page_wrap > div > div > a')
                last_page = int(a_tags[-1].text.strip())
            except:
                continue

            for start in range(1, last_page * 10, 10):
                params['start'] = start
                raw = requests.get(Url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)

                dom = BeautifulSoup(raw.text, "html.parser")

                lists = dom.select("#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li")

                for l in lists:
                    urls.append(l.select("div.news_wrap > div.news_area > div.news_info > div.info_group > a")[-1]['href'])
                
                print('page:', page, '/  code:', code, '/  num:', start, '\r', end='')
                page += 1
                time.sleep(1)

    # 기사 총 몇 개 클롤링 하는지 보여주기
    news_urls = [
         url
         for url in urls
         if 'https://news.naver.com/' in url
    ]
    return news_urls, params

def save_article(news_urls, params):
    # 크롤링한 날짜, 제목, 본문 저장
    news = []

    i = 1 # 몇 번째 기사를 크롤링하는지 체크
    
    tot_sum = 0 
    for url in tqdm(news_urls, desc = params['query']):
        raw = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

        dom = BeautifulSoup(raw.text, "html.parser")

        try:
            title = dom.select_one('#articleTitle').text.strip()
            date = dom.select_one('#main_content > div.article_header > div.article_info > div > span.t11').text.strip()[:10]
        except Exception:
            continue
        
        # <script> <style> 제거 (전처리)
        for s in dom(['script', 'style', 'img']):
            s.decompose()

        # 뉴스본문 리턴
        news_content = dom.find('div', attrs = {'id': 'articleBodyContents'})
        # 뉴스본문에 대한 전처리
        # 각 line 별로 strip(), 태그 제거
        lines = [
            line.strip()
            for line in news_content.get_text().splitlines()
        ]
        news_content = ''.join(lines).replace('[경향신문]','')

        dic = {
            'title' : title,
            'date' : date,
            'content' : news_content
        }
        news.append(dic)
        
        i += 1
        time.sleep(1)
        tot_sum += i

    df = pd.DataFrame(news)

    base_path = r'../../dataset'
    file_path = os.path.join(base_path, f"{params['query']}_기사_크롤링_2020.01.01_{params['de']}.csv")

    df.to_csv(file_path, encoding='utf-8-sig')
    

In [7]:

from sys import stdout
import time

# 진섭
keyword_list = [
 #'삼성SDI',
 #'삼성화재',
 #'두산중공업',
 #'CJ제일제당',
 #'강원랜드',
 #'GS건설',
 #'오리온',
 #'한국전력',
 #'현대글로비스',
 #'KT&G',
 #'SKC',
 #'삼성바이오로직스',
 #'LG유플러스',
 #'두산밥캣',
 #'유한양행',
 #'미래에셋증권',
 #'한국금융지주',
 #'삼성전기',
 #'삼성물산',
 #'SK',
 #'아모레퍼시픽',
 #'NH투자증권',
 #'셀트리온',
 #'GS리테일',
 #'삼성생명',
 #'삼성에스디에스',
 #'메리츠금융지주',
 #'현대모비스',
    
    #'GS',
    #'S-Oil'
    #'에스오일'
    'POSCO'
]

news_urls = []

cnt = 1

# 키워드 리스트
lst = keyword_list

for keyword in lst:
    print(cnt, "/", len(lst), keyword)
    news_urls, params = crawling_article(keyword)
    save_article(news_urls, params)
    cnt += 1

print('end')

1 / 1 POSCO


POSCO: 100%|█████████████████████████████████████████████████████████████████████████| 114/114 [02:34<00:00,  1.36s/it]

end


In [66]:
len(news_urls)

557

In [4]:
for i in keyword_list[:2]:
    print(i)

삼성SDI
삼성화재


In [8]:
# 전체 주식 정보 가져오기
import FinanceDataReader as fdr

df = fdr.StockListing("KRX")
df

# 다음 시가총액 TOP 100 읽어오기
import requests
from bs4 import BeautifulSoup

# 다음 시가총액 TOP 100
url = "https://finance.daum.net/api/trend/market_capitalization?page={}&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true"

headers={
    "User-Agent" : "Mozilla/5.0",
    "referer" : "https://finance.daum.net/domestic/market_cap?market=KOSPI"
}

stock_name = []
for i in range(1, 5):
    response = requests.get(url.format(i), headers=headers).json()  # json
    data = response['data']

    if i < 4:
        for j in range(30):
            stock_name.append(data[j]['name'])
    else:
        for j in range(10):
            stock_name.append(data[j]['name'])
            
# stock_name.extend(['포스코', '케이티', '케이티앤지', '에스케이바이오팜', '금호석유화학','삼성화재해상보험', '쌍용씨앤이', '아모레퍼시픽그룹', '한국전력공사', '현대자동차'])
print(stock_name)

stock_name_df = pd.DataFrame(stock_name)

# 시가총액 TOP 100 중 상장일이 2020-01-01 이전인 주식 정보 가져오기
stock_df = df[df['Name'].isin(stock_name)]
stock_df = stock_df[stock_df['ListingDate']<='2020-01-01']
stock_df['Name']

['삼성전자', 'SK하이닉스', 'NAVER', '삼성바이오로직스', '삼성전자우', '카카오', 'LG화학', '삼성SDI', '현대차', '기아', '카카오뱅크', '셀트리온', '카카오페이', 'POSCO', 'KB금융', '현대모비스', '크래프톤', '삼성물산', 'LG전자', '신한지주', 'SK이노베이션', 'SK', 'LG생활건강', 'SK바이오사이언스', '엔씨소프트', '한국전력', '삼성전기', '삼성생명', '하이브', 'HMM', '하나금융지주', 'LG', 'SK텔레콤', '삼성에스디에스', 'SK아이이테크놀로지', 'KT&G', '포스코케미칼', '두산중공업', '넷마블', '아모레퍼시픽', '대한항공', 'S-Oil', '현대중공업', '삼성화재', '우리금융지주', 'SK스퀘어', '고려아연', '기업은행', 'KT', 'LG디스플레이', '롯데케미칼', 'SK바이오팜', '한온시스템', 'SKC', '한국조선해양', '한화솔루션', 'LG이노텍', 'F&F', 'LG유플러스', '현대글로비스', '미래에셋증권', '현대제철', '코웨이', '현대건설', '맥쿼리인프라', '일진머티리얼즈', 'CJ제일제당', '금호석유', '에스디바이오센서', '강원랜드', 'KODEX 200', '한국타이어앤테크놀로지', '삼성중공업', '메리츠금융지주', '한국금융지주', '현대중공업지주', '삼성엔지니어링', '삼성증권', '유한양행', '두산밥캣', '이마트', '한진칼', '오리온', 'DB손해보험', '메리츠화재', '쌍용C&E', '삼성카드', 'NH투자증권', '아모레G', 'GS', '현대차2우B', '한미사이언스', '메리츠증권', '한솔케미칼', 'GS건설', '한국가스공사', '한전기술', 'GS리테일', '롯데지주', '한미약품']


63       CJ제일제당
77       DB손해보험
114          GS
116        GS건설
118       GS리테일
         ...   
7000     현대글로비스
7005      현대모비스
7027       현대제철
7029    현대중공업지주
7030        현대차
Name: Name, Length: 86, dtype: object

In [23]:
import random

stock_list = list(stock_df['Name'])
stock_list.pop(stock_list.index('삼성증권'))
random.shuffle(stock_list)  # 순서 바꿔주기
print(stock_list)
len(stock_list)

['일진머티리얼즈', 'LG화학', '메리츠화재', '삼성엔지니어링', '한국가스공사', '삼성중공업', '한화솔루션', '맥쿼리인프라', '카카오', 'DB손해보험', 'NAVER', '메리츠증권', '기아', '우리금융지주', '현대차', '이마트', '대한항공', '한솔케미칼', '아모레G', 'LG디스플레이', '한국타이어앤테크놀로지', '현대건설', '쌍용C&E', '한온시스템', '삼성전자', '금호석유', 'S-Oil', 'POSCO', 'GS', 'LG전자', '한진칼', 'LG생활건강', '고려아연', '한미사이언스', 'LG', '현대제철', '롯데케미칼', '포스코케미칼', '현대중공업지주', 'KT', 'SK하이닉스', '신한지주', 'LG이노텍', 'KB금융', '넷마블', '삼성카드', 'SK텔레콤', '한미약품', '엔씨소프트', '한국조선해양', 'HMM', '코웨이', '한전기술', '롯데지주', '기업은행', 'SK이노베이션', '하나금융지주', '삼성SDI', '삼성화재', '두산중공업', 'CJ제일제당', '강원랜드', 'GS건설', '오리온', '한국전력', '현대글로비스', 'KT&G', 'SKC', '삼성바이오로직스', 'LG유플러스', '두산밥캣', '유한양행', '미래에셋증권', '한국금융지주', '삼성전기', '삼성물산', 'SK', '아모레퍼시픽', 'NH투자증권', '셀트리온', 'GS리테일', '삼성생명', '삼성에스디에스', '메리츠금융지주', '현대모비스']


85

In [24]:
# 하병노
stock_list[:29] 

'''
['일진머티리얼즈',
 'LG화학',
 '메리츠화재',
 '삼성엔지니어링',
 '한국가스공사',
 '삼성중공업',
 '한화솔루션',
 '맥쿼리인프라',
 '카카오',
 'DB손해보험',
 'NAVER',
 '메리츠증권',
 '기아',
 '우리금융지주',
 '현대차',
 '이마트',
 '대한항공',
 '한솔케미칼',
 '아모레G',
 'LG디스플레이',
 '한국타이어앤테크놀로지',
 '현대건설',
 '쌍용C&E',
 '한온시스템',
 '삼성전자',
 '금호석유',
 'S-Oil',
 'POSCO',
 'GS']
 '''

['일진머티리얼즈',
 'LG화학',
 '메리츠화재',
 '삼성엔지니어링',
 '한국가스공사',
 '삼성중공업',
 '한화솔루션',
 '맥쿼리인프라',
 '카카오',
 'DB손해보험',
 'NAVER',
 '메리츠증권',
 '기아',
 '우리금융지주',
 '현대차',
 '이마트',
 '대한항공',
 '한솔케미칼',
 '아모레G',
 'LG디스플레이',
 '한국타이어앤테크놀로지',
 '현대건설',
 '쌍용C&E',
 '한온시스템',
 '삼성전자',
 '금호석유',
 'S-Oil',
 'POSCO',
 'GS']

In [25]:
# 김영은
stock_list[29:57]

'''
['LG전자',
 '한진칼',
 'LG생활건강',
 '고려아연',
 '한미사이언스',
 'LG',
 '현대제철',
 '롯데케미칼',
 '포스코케미칼',
 '현대중공업지주',
 'KT',
 'SK하이닉스',
 '신한지주',
 'LG이노텍',
 'KB금융',
 '넷마블',
 '삼성카드',
 'SK텔레콤',
 '한미약품',
 '엔씨소프트',
 '한국조선해양',
 'HMM',
 '코웨이',
 '한전기술',
 '롯데지주',
 '기업은행',
 'SK이노베이션',
 '하나금융지주']
'''

['LG전자',
 '한진칼',
 'LG생활건강',
 '고려아연',
 '한미사이언스',
 'LG',
 '현대제철',
 '롯데케미칼',
 '포스코케미칼',
 '현대중공업지주',
 'KT',
 'SK하이닉스',
 '신한지주',
 'LG이노텍',
 'KB금융',
 '넷마블',
 '삼성카드',
 'SK텔레콤',
 '한미약품',
 '엔씨소프트',
 '한국조선해양',
 'HMM',
 '코웨이',
 '한전기술',
 '롯데지주',
 '기업은행',
 'SK이노베이션',
 '하나금융지주']

In [26]:
# 김진섭
stock_list[57:]

'''
['삼성SDI',
 '삼성화재',
 '두산중공업',
 'CJ제일제당',
 '강원랜드',
 'GS건설',
 '오리온',
 '한국전력',
 '현대글로비스',
 'KT&G',
 'SKC',
 '삼성바이오로직스',
 'LG유플러스',
 '두산밥캣',
 '유한양행',
 '미래에셋증권',
 '한국금융지주',
 '삼성전기',
 '삼성물산',
 'SK',
 '아모레퍼시픽',
 'NH투자증권',
 '셀트리온',
 'GS리테일',
 '삼성생명',
 '삼성에스디에스',
 '메리츠금융지주',
 '현대모비스']
'''

['삼성SDI',
 '삼성화재',
 '두산중공업',
 'CJ제일제당',
 '강원랜드',
 'GS건설',
 '오리온',
 '한국전력',
 '현대글로비스',
 'KT&G',
 'SKC',
 '삼성바이오로직스',
 'LG유플러스',
 '두산밥캣',
 '유한양행',
 '미래에셋증권',
 '한국금융지주',
 '삼성전기',
 '삼성물산',
 'SK',
 '아모레퍼시픽',
 'NH투자증권',
 '셀트리온',
 'GS리테일',
 '삼성생명',
 '삼성에스디에스',
 '메리츠금융지주',
 '현대모비스']